In [1]:
%pip install pytorch-lightning
%pip install torchinfo
%pip install numpy==1.21.6
%pip install scikit-learn==1.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 74.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
flax 0.7.5 requires numpy>=1.22, but you have numpy 1.21.6 which is incompatible.
jax 0.4.20 requires numpy>=1.22, but you have numpy 1.21.6 which is incompatible.
jaxlib 0.4.20+cuda11.cudnn86 requires n

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 which is incompatible.


In [2]:
import torch
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from torch.nn import Conv2d, ConvTranspose2d, Flatten, Linear, LeakyReLU, Sequential, MSELoss
import numpy as np
from sklearn.cluster import KMeans
import random
from skimage.transform import resize
from torchvision import datasets, transforms
import math
import matplotlib.pyplot as plt
import torchvision.transforms.functional
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import FashionMNIST
import sklearn.metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression

In [3]:
def kmeans(samples):
  k = 3
  for i in range(1):
    N = []
    A = []
    N_samp = []
    for j in range(54000):
      if samples[j][1] == i:
        N.append(samples[j][0])
      else:
        x = samples[j][0].squeeze(0).numpy().flatten()
        A.append(x)
    A1 = []
    A2 = []
    A3 = []
    N_samp = random.sample(N, 3000)
    kmeans = KMeans(n_clusters=k).fit(A)
    for i in range(len(kmeans.labels_)):
        if kmeans.labels_[i] == 0:
          A1.append(np.reshape(np.array(A[i]), (32, 32)))
        if kmeans.labels_[i] == 1:
          A2.append(np.reshape(np.array(A[i]), (32, 32)))
        if kmeans.labels_[i] == 2:
          A3.append(np.reshape(np.array(A[i]), (32, 32)))
    A1_sample_index = random.sample(range(1, len(A1)), 1000)
    A2_sample_index = random.sample(range(1, len(A2)), 1000)
    A3_sample_index = random.sample(range(1, len(A3)), 1000)
    A1_samp = [[A1[indx], 0] for indx in A1_sample_index]
    A2_samp = [[A2[indx], 0] for indx in A2_sample_index]
    A3_samp = [[A3[indx], 0] for indx in A3_sample_index]
    N_samp = [[np.reshape(np.array(img), (32, 32)), 1] for img in N_samp]
    return N_samp + A1_samp + A2_samp + A3_samp

In [4]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./"):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([transforms.Resize((32, 32)),
                                             transforms.ToTensor()])

    def prepare_data(self):
        # download
        FashionMNIST(self.data_dir, train=True, download=True)
        FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage: str):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            mnist_full = FashionMNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [0.90, 0.10])
            self.mnist_val, _ = random_split(self.mnist_val, [250, 5750])
            self.mnist_train = kmeans(self.mnist_train)

        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.mnist_test = FashionMNIST(self.data_dir, train=False,
                                           transform=self.transform)

        if stage == "predict":
            self.mnist_predict = FashionMNIST(self.data_dir, train=False,
                                              transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=1, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=1, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=1, shuffle=False)


dm = MNISTDataModule()

In [5]:
class AutoEncoder(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=1, out_channels=22,
                            kernel_size=4, stride=2, padding=1)
        self.conv2 = Conv2d(in_channels=22, out_channels=6,
                            kernel_size=4, stride=2, padding=1)
        self.conv3 = Conv2d(in_channels=6, out_channels=84,
                            kernel_size=4, stride=2, padding=1)
        self.conv4 = Conv2d(in_channels=84, out_channels=24,
                            kernel_size=4, stride=2, padding=1)
        self.up_conv1 = ConvTranspose2d(in_channels=24, out_channels=84,
                                        kernel_size=4, stride=2, padding=1)
        self.up_conv2 = ConvTranspose2d(in_channels=168, out_channels=6,
                                        kernel_size=4, stride=2, padding=1)
        self.up_conv3 = ConvTranspose2d(in_channels=12, out_channels=22,
                                        kernel_size=4, stride=2, padding=1)
        self.up_conv4 = ConvTranspose2d(in_channels=44, out_channels=1,
                                        kernel_size=4, stride=2, padding=1)
        self.automatic_optimization = False
        self.l_relu = LeakyReLU()

    def forward(self, x):
        c1 = self.l_relu(self.conv1(x))
        c2 = self.l_relu(self.conv2(c1))
        c3 = self.l_relu(self.conv3(c2))
        c4 = self.l_relu(self.conv4(c3))
        u_c1 = self.l_relu(self.up_conv1(c4))
        u_c1 = torch.cat((u_c1,c3), 0)
        u_c2 = torch.relu(self.up_conv2(u_c1))
        u_c2 = torch.cat((u_c2,c2), 0)
        u_c3 = torch.relu(self.up_conv3(u_c2))
        u_c3 = torch.cat((u_c3,c1), 0)
        u_c4 = torch.relu(self.up_conv4(u_c3))
        return u_c4

In [6]:
class Net(LightningModule):
    def __init__(self):
      super().__init__()
      self.save_hyperparameters()
      self.autoencoder = AutoEncoder()
      self.loss_fct = MSELoss(reduction='mean')
      self.validation_step_outputs = []
      self.errors = []
      self.labels = []
      self.log_reg = LogisticRegression(solver="newton-cg")


    def forward(self, x):
        return self.autoencoder(x)
        """z = self.encoder(x)
        print(z.size())
        return self.decoder(z)"""

    def training_step(self, sample, batch_idx):
        x, normal = sample
        x_reconstructed = self.forward(x)
        loss = self.loss_fct(x, x_reconstructed)
        if normal:
          optimizer = self.optimizers()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          self.errors.append(loss)
          self.labels.append(1)
        else:
          self.errors.append(loss)
          self.labels.append(0)


    def validation_step(self, batch, batch_idx):
        if not self.trainer.sanity_checking:
            x, y = batch
            x = x.squeeze(0)
            losses = []
            if y != 1:
              y = 0
              # Adding noise to anomaly images
              #x += torch.randn(x.size()) * 0.3
            else:
              y = 1
            x_reconstructed = self.forward(x)
            loss = self.loss_fct(x, x_reconstructed)
            pred = {"label": y,
                    "recon_error": loss}
            self.validation_step_outputs.append(pred)
            return pred

    def on_validation_epoch_end(self):
        if not self.trainer.sanity_checking:
            y = []
            y_hat = []
            errors = []
            self.log_reg.fit(np.array(self.errors).reshape(-1, 1), np.array(self.labels))
            y_pred = []
            y_true = []
            for x in self.validation_step_outputs:
                y_hat = self.log_reg.predict_proba(np.array(x['recon_error']).reshape(-1, 1))
                y_pred.append(y_hat[0][1])
                y_true.append(x['label'])
            self.auroc = roc_auc_score(np.array(y_true), np.array(y_pred))
            print("Auorc: ", self.auroc)
            self.log_dict({"auroc": self.auroc})
            self.validation_step_outputs.clear()
            self.errors = []
            self.labels = []
            self.log_reg = LogisticRegression(solver="newton-cg")


    # Results :
    # Predicts 1 always... maybe we have to add noise to anomaly data
    # Errors are very small, below 0.1

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-3)
        return optimizer

In [7]:

pl.seed_everything(99, workers=True)
net = Net()
net.train()
trainer = pl.Trainer(
    deterministic = True,
    accelerator = "auto",
    devices = 1,
    max_epochs = 100)
trainer.fit(net, dm)

INFO:lightning_fabric.utilities.seed:Seed set to 99
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


100%|██████████| 26421880/26421880 [00:01<00:00, 18225311.97it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 306025.30it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



...

Validation: |          | 0/? [00:00<?, ?it/s]

Auorc:  0.6163127413127414


Validation: |          | 0/? [00:00<?, ?it/s]

Auorc:  0.6592664092664093


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
